<a href="https://colab.research.google.com/github/man63u/-/blob/main/Vesper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装 PyTorch 和 Hugging Face Transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install soundfile  # 用于加载音频文件

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.

In [30]:
# 克隆 Vesper 仓库
!git clone https://github.com/HappyColor/Vesper.git
!cd Vesper

# 下载预训练模型（假设模型权重文件为 vesper-4.pth 或 vesper-12.pth）
# 如果没有提供模型权重，可以使用 Hugging Face 的 WavLM 作为替代
!wget https://raw.githubusercontent.com/HappyColor/Vesper/refs/heads/master/configs/model_config.py  # 替换为实际下载链接

Cloning into 'Vesper'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 50 (delta 8), reused 45 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 154.63 KiB | 17.18 MiB/s, done.
Resolving deltas: 100% (8/8), done.
--2025-03-08 10:49:33--  https://raw.githubusercontent.com/HappyColor/Vesper/refs/heads/master/configs/model_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2477 (2.4K) [text/plain]
Saving to: ‘model_config.py’

model_config.py     100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-03-08 10:49:33 (37.6 MB/s) - ‘model_config.py’ saved [2477/2477]



In [35]:
# %load dataset_config.py

from yacs.config import CfgNode as CN

_C = CN(new_allowed=True)

###########
# IEMOCAP #
###########
_C.iemocap = CN(new_allowed=True)
_C.iemocap.num_classes = 4
_C.iemocap.meta_csv_file = '/148Dataset/data-chen.weidong/iemocap/feature/name_label_text.csv'
_C.iemocap.wavdir = '/148Dataset/data-chen.weidong/iemocap/wav_all_sentences'
_C.iemocap.batch_length = 104000 # 16000 * 6.5
_C.iemocap.evaluate = ['accuracy', 'recall']
_C.iemocap.folds = [1, 2, 3, 4, 5]
_C.iemocap.f1 = 'weighted'
_C.iemocap.have_test_set = False

########
# MELD #
########
_C.meld = CN(new_allowed=True)
_C.meld.num_classes = 7
_C.meld.meta_csv_file = '/148Dataset/data-chen.weidong/meld/label/official'
_C.meld.wavdir = '/148Dataset/data-chen.weidong/meld/audio_16k'
_C.meld.batch_length = 72000 # 16000 * 4.5
_C.meld.evaluate = ['f1']
_C.meld.folds = [1]
_C.meld.f1 = 'weighted'
_C.meld.have_test_set = True

###########
# CREMA-D #
###########
_C.crema = CN(new_allowed=True)
_C.crema.num_classes = 6
_C.crema.meta_csv_file = '/148Dataset/data-chen.weidong/CREMA-D/CREMA-D.csv'
_C.crema.wavdir = '/148Dataset/data-chen.weidong/CREMA-D/AudioWAV'
_C.crema.batch_length = 48000 # 16000 * 3.0
_C.crema.evaluate = ['accuracy', 'recall']
_C.crema.folds = [1]
_C.crema.f1 = 'weighted'
_C.crema.have_test_set = False

#########
# LSSED #
#########
_C.lssed = CN(new_allowed=True)
_C.lssed.num_classes = 4
_C.lssed.meta_csv_file = '/148Dataset/data-chen.weidong/lssed_all/metadata_english_all.csv'
_C.lssed.wavdir = '/148Dataset/data-chen.weidong/lssed_all/wav_all'
_C.lssed.batch_length = 80000 # 16000*5
_C.lssed.evaluate = ['accuracy', 'recall']
_C.lssed.folds = [1]
_C.lssed.f1 = 'weighted'
_C.lssed.have_test_set = True

_C.lssed.target_length = 249
_C.lssed.l_target_dir = '/148Dataset/data-chen.weidong/lssed_all/feature/wavlm_large_L12_mat'
_C.lssed.h_target_dir = '/148Dataset/data-chen.weidong/lssed_all/feature/wavlm_large_L24_mat'


In [36]:
# 创建新文件夹（如 AudioMP3）
!mkdir /content/wavdir

In [38]:
# 移动所有 MP3 文件到 AudioMP3 文件夹
!mv /content/*.mp3 /content/wavdir/

In [39]:
import os
import pandas as pd

# 音频文件目录
audio_dir = "/content/wavdir"

# 列出所有音频文件
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

# 创建元数据
data = {"name": audio_files}
df = pd.DataFrame(data)

# 保存为 CSV 文件
df.to_csv("/content/meta_csv_file.csv", index=False)

print("元数据文件已生成：/content/meta_csv_file.csv")

元数据文件已生成：/content/meta_csv_file.csv


In [ ]:
# 带标签元数据
import os
import pandas as pd

# 音频文件目录
audio_dir = "/content/wavdir"

# 列出所有音频文件
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

# 解析文件名生成标签
data = []
for file in audio_files:
    # 假设文件名格式为 "audio1_anger.wav"
    name, label = file.rsplit("_", 1)  # 从右边分割一次
    label = label.replace(".wav", "")  # 去掉文件扩展名
    data.append({"name": file, "label": label})

# 创建 DataFrame
df = pd.DataFrame(data)

# 保存为 CSV 文件
df.to_csv("/content/metadata_english_all.csv", index=False)

print("元数据文件已生成：/content/metadata_english_all.csv")

In [51]:
# 改变文件为可写模式
!chmod 777 /content/Vesper/extract_feature/WavLM/extract_wavlm.py


提取预训练数据特征

In [53]:
import torch

ckpt_path = "/content/pytorch_model.bin"

# 检查文件是否存在且非空
if not os.path.exists(ckpt_path) or os.path.getsize(ckpt_path) == 0:
    raise FileNotFoundError(f"权重文件 {ckpt_path} 不存在或为空，请重新下载。")

# 尝试加载
checkpoint = torch.load(ckpt_path, map_location="cpu")

# 查看 checkpoint 的 keys
print(checkpoint.keys())


<ipython-input-53-9b8ee5cb1e69>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")


odict_keys(['masked_spec_embed', 'feature_extractor.conv_layers.0.conv.weight', 'feature_extractor.conv_layers.0.layer_norm.weight', 'feature_extractor.conv_layers.0.layer_norm.bias', 'feature_extractor.conv_layers.1.conv.weight', 'feature_extractor.conv_layers.2.conv.weight', 'feature_extractor.conv_layers.3.conv.weight', 'feature_extractor.conv_layers.4.conv.weight', 'feature_extractor.conv_layers.5.conv.weight', 'feature_extractor.conv_layers.6.conv.weight', 'feature_projection.layer_norm.weight', 'feature_projection.layer_norm.bias', 'feature_projection.projection.weight', 'feature_projection.projection.bias', 'encoder.pos_conv_embed.conv.bias', 'encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v', 'encoder.layer_norm.weight', 'encoder.layer_norm.bias', 'encoder.layers.0.attention.gru_rel_pos_const', 'encoder.layers.0.attention.k_proj.weight', 'encoder.layers.0.attention.k_proj.bias', 'encoder.layers.0.attention.v_proj.weight', 'encoder.layers.0.attention.

In [54]:
from transformers import WavLMModel, WavLMConfig

# 使用 Hugging Face 加载模型和配置
model = WavLMModel.from_pretrained("/content")


In [52]:
!python /content/Vesper/extract_feature/WavLM/extract_wavlm.py

Traceback (most recent call last):
  File "/content/Vesper/extract_feature/WavLM/extract_wavlm.py", line 119, in <module>
    main(args)
  File "/content/Vesper/extract_feature/WavLM/extract_wavlm.py", line 83, in main
    cfg = WavLMConfig(checkpoint['cfg'])
                      ~~~~~~~~~~^^^^^^^
KeyError: 'cfg'


In [46]:
!ls -lh /content/Vesper/extract_feature/WavLM/WavLM-Base+.pt


-rw-r--r-- 1 root root 0 Mar  8 12:08 /content/Vesper/extract_feature/WavLM/WavLM-Base+.pt


In [32]:
# %load /content/model_config.py

from yacs.config import CfgNode as CN
_C = CN(new_allowed=True)

###############
# Transformer #
###############
_C.Transformer = CN(new_allowed=True)

_C.Transformer.num_encoder_layers = 4
_C.Transformer.embed_dim = 1024
_C.Transformer.ffn_embed_dim = 512
_C.Transformer.num_heads = 8
_C.Transformer.activation = 'gelu'
_C.Transformer.dropout = 0.1
_C.Transformer.bias = True
_C.Transformer.normalize_before = True

# positional embeddings
_C.Transformer.conv_pos = 128
_C.Transformer.conv_pos_groups = 16

##########
# Vesper #
##########
_C.Vesper = CN(new_allowed=True)

# mainstream model
_C.Vesper.encoder_layers= 4
_C.Vesper.encoder_embed_dim = 1024
_C.Vesper.ffn_embed_dim = 4096
_C.Vesper.num_heads = 16
_C.Vesper.activation = 'gelu'
_C.Vesper.dropout = 0.1
_C.Vesper.bias = True
_C.Vesper.normalize = True
_C.Vesper.normalize_before = True
_C.Vesper.relative_position_embedding = True
_C.Vesper.qk_norm = False  # query/key (QK) normalization

# predictor
_C.Vesper.enable_l_predictor = True
_C.Vesper.enable_h_predictor = True
_C.Vesper.enable_x_predictor = True

# FinetuneWrapper
_C.Vesper.projector_dim = 256
_C.Vesper.output_rep = 'weighted_sum' # 'weighted_sum' / 'last_layer'

# initiliaze with wavlm
_C.Vesper.init_with_wavlm = True
_C.Vesper.init_style = ['uniform_extract']   # ['custom_average', [(0, 1), (2, 5), (6, 13), (14, 23)]], ['custom_extract', [0, 5, 11, 17]]
_C.Vesper.path_to_wavlm = ['/148Dataset/data-chen.weidong/pre_trained_model/wavlm/WavLM-Base.pt', '/148Dataset/data-chen.weidong/pre_trained_model/wavlm/WavLM-Large.pt']

# initiliaze with other pre-trained model
_C.Vesper.init_with_ckpt = False
_C.Vesper.path_to_vesper = ''

# rms-based mask
_C.Vesper.mask_depend_on_rms = True
_C.Vesper.frame_length = 400 # 16000 * 0.025
_C.Vesper.hop_length = 320   # 16000 * 0.020
_C.Vesper.span_space = 1
_C.Vesper.h_up = 1.0
_C.Vesper.h_down = 0.5
_C.Vesper.l_up = 0.49
_C.Vesper.l_down = 0.2
_C.Vesper.small_span = 8
_C.Vesper.num_small_span = 20
_C.Vesper.large_span = 40
_C.Vesper.num_large_span = 4
_C.Vesper.max_mask_percentage = 0.64

# positional embedding
_C.Vesper.conv_pos = 128
_C.Vesper.conv_pos_groups = 16

# bucket relative position embedding
_C.Vesper.num_buckets = 320
_C.Vesper.max_distance = 800
_C.Vesper.gru_rel_pos = True

# feature encoder
_C.Vesper.extractor_mode = 'layer_norm'   # 'default' / 'layer_norm'
_C.Vesper.conv_feature_layers = '[(512,10,5)] + [(512,3,2)] * 4 + [(512,2,2)] * 2'
_C.Vesper.dropout_input = 0.0


In [4]:
import torch
from transformers import Wav2Vec2FeatureExtractor, WavLMForSequenceClassification
import soundfile as sf  # 用于加载音频文件

# 加载 Vesper 模型（假设基于 WavLM）
model_path = "path/to/vesper-4"  # 替换为实际模型路径
model = WavLMForSequenceClassification.from_pretrained(model_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)

# 加载音频文件
def load_audio(file_path):
    audio_input, sampling_rate = sf.read(file_path)
    return audio_input, sampling_rate

# 测试音频文件路径
audio_file = "path/to/test_audio.wav"  # 替换为实际音频文件路径
audio_input, sampling_rate = load_audio(audio_file)

# 提取特征
inputs = feature_extractor(audio_input, sampling_rate=sampling_rate, return_tensors="pt")

# 推理
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

# 输出情绪类别
emotion_classes = ["anger", "happy", "sad", "neutral"]  # 假设情绪类别
print(f"Predicted emotion: {emotion_classes[predicted_class]}")

RuntimeError: Failed to import transformers.models.wavlm.modeling_wavlm because of the following error (look up to see its traceback):
Detected that PyTorch and torchvision were compiled with different CUDA major versions. PyTorch has CUDA Version=11.8 and torchvision has CUDA Version=12.4. Please reinstall the torchvision that matches your PyTorch install.

In [10]:
# 列出 sample_data 目录下的文件
!ls /content

model_config.py  sample_data


In [ ]:
# 移动文件到工作目录
!mv /content/sample_data/dataset_config.py /content/

In [8]:
!ls /content/

model_config.py  sample_data


In [9]:
!cat /content/dataset_config.py

cat: /content/dataset_config.py: No such file or directory


In [ ]:
from google.colab import files

# 上传音频文件
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]  # 获取上传的文件名

运行推理

In [ ]:
# 加载音频文件
audio_input, sampling_rate = load_audio(audio_file)

# 提取特征
inputs = feature_extractor(audio_input, sampling_rate=sampling_rate, return_tensors="pt")

# 推理
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

# 输出情绪类别
print(f"Predicted emotion: {emotion_classes[predicted_class]}")